In [1]:
import torch
import numpy as np
import torch.nn as nn
import os
import sys

In [2]:
notebook_path = os.getcwd()
parent_dir = os.path.dirname(notebook_path)
sys.path.append(parent_dir)

In [3]:
import importnb
with __import__('importnb').Notebook(): 
    from tools import ScaledDotProductAttention
    from tools import MultiHeadAttention
    from tools import AddPositionalEncoding
    from tools import TransformerFFN
    from notebooks.Transformer import TransformerEncoderLayer

## SelfAttentionの実行

### 適当なデータの作成
---
新しくデータkを作る。kは***(B,T,d_model)***のshapeを持つテンソルである。


### SelfAttentionにする
---
q,k,vを同じテンソルにすることでSelfAttentionにする。


### forwardで計算を行う。
---
呼び出したspa.forward()によって計算を行う。
この時spaの初期化で与えるd_kには本来d_modelをhead数で割った値が入る(デフォルトだと512/8で64)

In [4]:
###SelfAttentionの実行
num_head = 4
batch_size = 2
seq_len = 10
d_model = 64
max_len = 512
d_ff =1024
dropout_rate=0.1
layer_norm_eps = 1e-5
k = torch.randn(batch_size,seq_len,d_model)

In [6]:
###SelfAttentionにするためにqとvとkを一緒にする
pe = AddPositionalEncoding(d_model,max_len)
k = pe(k)
q = k
v = k
spa = ScaledDotProductAttention(16)
attention_weight = spa.forward(q,k,v)

In [7]:
attention_weight.shape

torch.Size([2, 10, 64])

## MultiHeadAttentionの実行
ランダムなテンソル、スタンダードなマスクを使う。

In [7]:
def create_incremental_mask(seq_len):
    """
    seq_len x seq_len のサイズのマスクを生成する。
    0列目は全てFalse、以降の列では上から順にTrueの数を増やしていく。
    """
    # seq_len x seq_len の行列を生成し、初期値は全てFalseに設定
    mask = torch.full((seq_len, seq_len), False)

    # 各列に対して、上から順にTrueをセットする
    for i in range(seq_len):
        mask[:i, i] = True

    return mask
mask = create_incremental_mask(seq_len).repeat(batch_size,1,1)
mask.shape

torch.Size([2, 10, 10])

In [9]:
mha = MultiHeadAttention(num_head,d_model)
output = mha(q,k,v,mask=mask)

In [5]:
encoder = TransformerEncoderLayer(
    d_model = d_model,
    d_ff = d_ff,
    num_head = num_head,
    dropout_rate=dropout_rate,
    layer_norm_eps=layer_norm_eps
)

In [8]:
output = encoder(k,mask=mask)

In [9]:
output.shape

torch.Size([2, 10, 64])

In [10]:
output

tensor([[[-0.4798, -1.6409,  1.5316,  ..., -0.2062, -0.6141,  0.1137],
         [-1.0895,  1.2770, -0.3501,  ..., -0.1438, -0.0943, -0.3211],
         [-0.7535,  1.0697, -0.3661,  ..., -0.3552,  1.0728, -2.2408],
         ...,
         [-0.5272, -1.3278,  1.1327,  ..., -0.6883,  2.1593,  0.5936],
         [ 2.5874,  0.0259,  0.3282,  ...,  2.3327, -1.8902,  0.8085],
         [-1.2454,  1.3842,  0.2644,  ...,  0.5179,  0.8128,  0.8103]],

        [[-0.0745, -1.6565, -0.4859,  ..., -0.6737,  1.6863, -1.8581],
         [ 1.6795, -1.8668, -1.4459,  ...,  0.4184,  0.3278, -0.5743],
         [-0.1570, -0.5188,  0.5838,  ...,  1.3809, -1.4716, -0.3097],
         ...,
         [ 1.1120, -0.7480,  0.8193,  ..., -0.7944,  1.2841,  1.1541],
         [ 1.2012, -1.2838, -0.8361,  ..., -0.5563, -0.9324, -0.2621],
         [-0.4867,  0.6821,  0.0384,  ...,  0.0147,  2.3731,  0.3057]]],
       grad_fn=<NativeLayerNormBackward0>)